In [39]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
from datetime import datetime

In [241]:
df = pd.read_csv("mpk_data.csv")
datetime_format = "%H:%M:%S"
df.head()

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_20016/2217797963.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("mpk_data.csv")


,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [ ]:
start_stops = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].rename(
    columns={'start_stop': 'name', 'start_stop_lat': 'lat', 'start_stop_lon': 'lon'}
)

end_stops = df[['end_stop', 'end_stop_lat', 'end_stop_lon']].rename(
    columns={'end_stop': 'name', 'end_stop_lat': 'lat', 'end_stop_lon': 'lon'}
)

all_stops = pd.concat([start_stops, end_stops]).drop_duplicates()
all_stops['index'] = all_stops.index + 1

df = df.merge(
    all_stops.rename(columns={'name': 'start_stop', 'lat': 'start_stop_lat', 'lon': 'start_stop_lon'}),
    on=['start_stop', 'start_stop_lat', 'start_stop_lon'],
    how='left'
)
df = df.rename(columns={'index': 'start_stop_index'})

df = df.merge(
    all_stops.rename(columns={'name': 'end_stop', 'lat': 'end_stop_lat', 'lon': 'end_stop_lon'}),
    on=['end_stop', 'end_stop_lat', 'end_stop_lon'],
    how='left'
)
df = df.rename(columns={'index': 'end_stop_index'})

In [267]:
all_stops

,name,lat,lon,index
0,Zajezdnia Obornicka,51.148737,17.021069,1
1,Paprotna,51.147752,17.020539,2
2,Obornicka (Wołowska),51.144385,17.023735,3
3,Bezpieczna,51.141360,17.026376,4
4,Bałtycka,51.136632,17.030617,5
...,...,...,...,...
946554,Żórawina - Niepodległości (Mostek),50.979741,17.040690,946555
978616,Smolec - Wiśniowa,51.087869,16.909206,978617
987746,Zabrodzie - pętla,51.056930,16.946521,987747
990859,Godzieszowa - transformator,51.241112,17.130596,990860


In [245]:
df.head()

,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,start_stop_index,end_stop_index
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,1,2
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,2,3
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,3,4
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,4,5
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,5,6


In [246]:
nan_rows = df[df.isna().any(axis=1)]
nan_rows

,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,start_stop_index,end_stop_index


In [253]:
def correct_time(time_str):
    if time_str.startswith('24:'):
        return '00' + time_str[2:]
    elif time_str.startswith('25:'):
        return '01' + time_str[2:]
    elif time_str.startswith('26:'):
        return '02' + time_str[2:]
    else:
        return time_str

df['departure_time'] = df['departure_time'].apply(correct_time)
df['arrival_time'] = df['arrival_time'].apply(correct_time)

In [254]:
df.to_csv('mpk_indexed.csv')
df = pd.read_csv('mpk_indexed.csv')

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_20016/2475752914.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('mpk_indexed.csv')


In [ ]:
@dataclass
class Connection:
    target_obj: 'BusStop'
    target_name: str
    line_name: str
    departure_t: datetime
    arrival_at: datetime

class BusStop:
    def __init__(self, latitude: float, longitude: float, 
                 name: str, index: int,
                 connections: List[Connection] = None):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.index = index
        self.connections = connections if connections is not None else []

    def __str__(self):
        return f'{self.name} {self.latitude} {self.longitude}'


In [255]:
df_unique = df.drop_duplicates(subset=['start_stop_index'])
df_unique.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,start_stop_index,end_stop_index
0,0,0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,1,2
1,1,1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,2,3
2,2,2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,3,4
3,3,3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,4,5
4,4,4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,5,6


In [256]:
connection_graph: List[BusStop] = []

for _, row in df_unique.iterrows():
    bus_stop = BusStop(
        name=row['start_stop'],
        latitude=row['start_stop_lat'],
        longitude=row['start_stop_lon'],
        index=row['start_stop_index']
    )

    connection_graph.append(bus_stop)

In [258]:
get_bus_stop = lambda x: [bs for bs in connection_graph if bs.index == x]

for _, row in df.iterrows():
    try:
        connection = Connection(
            target_name=row['end_stop'],
            line_name=row['line'],
            departure_t=datetime.strptime(
                row['departure_time'], datetime_format
            ),
            arrival_at=datetime.strptime(
                row['arrival_time'], datetime_format
            ),
            target_obj=get_bus_stop(row['end_stop_index'])[0]
        )
        
        start_stop = get_bus_stop(row['start_stop_index'])[0]
        start_stop.connections.append(connection)
    except Exception as e:
        print(row)

Unnamed: 0.2              154842
Unnamed: 0.1              154842
Unnamed: 0                154842
company             MPK Tramwaje
line                           8
departure_time          05:45:00
arrival_time            05:48:00
start_stop            Trzebnicka
end_stop               KARŁOWICE
start_stop_lat         51.128134
start_stop_lon         17.036664
end_stop_lat           51.134017
end_stop_lon           17.036712
start_stop_index           54024
end_stop_index            154843
Name: 154842, dtype: object
Unnamed: 0.2              154888
Unnamed: 0.1              154888
Unnamed: 0                154888
company             MPK Tramwaje
line                           8
departure_time          09:26:00
arrival_time            09:29:00
start_stop            Trzebnicka
end_stop               KARŁOWICE
start_stop_lat         51.128134
start_stop_lon         17.036664
end_stop_lat           51.134017
end_stop_lon           17.036712
start_stop_index           54024
end_stop_index 

KeyboardInterrupt: 

In [266]:
get_bus_stop(54024)[0].name

'Trzebnicka'